### Import Dependencies

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.utils import np_utils

#### avoid oom errors by setting gpu memory consumption growth

In [ ]:
# grab all the gpus available in the machine
gpus = tf.config.experimental.list_physical_devices('GPU')
# for every gpu set memory growth (making tensorflow to keep the memory only to what it needs)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Dataset

In [ ]:
# get the images and labels from file
with h5py.File('dataset/Galaxy10_DECals.h5', 'r') as F:
    images = np.array(F['images'])
    labels = np.array(F['ans'])

# convert the labels to categorical 10 classes
labels = np_utils.to_categorical(labels, 10)

# convert to desirable type
labels = labels.astype(np.float32)
images = images.astype(np.float32)